# TensorFlow 入門（覚え書き）

## 1. データフローグラフ
データの流れ（フロー）をグラフ（ネットワーク）として表現したもの。
ノード（点）とエッジ（線）からなる。
ノードとしては、定数、変数、（TensorFlow の）演算子などがある。

## 2. 定数、変数
TensorFlow では、定数や変数は専用のメソッドで定義する。オブジェクト指向されていない代数処理をイメージすると分かりやすい。

In [1]:
import tensorflow as tf

In [2]:
# 定数の定義
a = tf.constant(1, name='a')
b = tf.constant(2, name='b')

# ためしに print() してみる
print(a)
print(b)

# 定数は四則演算すると演算子のノードとなる
c = a + b
print(c)

Tensor("a:0", shape=(), dtype=int32)
Tensor("b:0", shape=(), dtype=int32)
Tensor("add:0", shape=(), dtype=int32)


c には、a + b という式が入っているとイメージすればよい。

In [3]:
# 変数の定義
d = tf.Variable(0, name='d') # 0 で初期化
print(d)

<tf.Variable 'd:0' shape=() dtype=int32_ref>


変数に値（や式）を設定するには、メソッド（assign）を利用する必要がある。

In [4]:
e = tf.assign(d, a + b) # e に d = a + b という式が入っている
print(e)

Tensor("Assign:0", shape=(), dtype=int32_ref)


加算や乗算などの演算をメソッド（add, multiply）で実行することができる。

In [5]:
f = tf.add(a, b)
g = tf.multiply(a, b)
print(f)
print(g)

Tensor("Add:0", shape=(), dtype=int32)
Tensor("Mul:0", shape=(), dtype=int32)


## 3. セッション
TensorFlow で計算結果（値の表示も含む）を得るためにはセッションを作成し、実行する必要がある。

In [6]:
with tf.Session() as sess:
    print('a =', sess.run(a))
    print('b =', sess.run(b))
    print('c =', sess.run(c))

a = 1
b = 2
c = 3


セッションで変数を利用する場合には、初期化が必要

In [7]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('d =', sess.run(d)) # まだ e が評価されていないので 0 となる
    print('e =', sess.run(e))
    # run(e) の後には、d に a + b が入っている
    print('d =', sess.run(d))
    # 演算の実行
    print('f =', sess.run(f))
    print('g =', sess.run(g))

d = 0
e = 3
d = 3
f = 3
g = 2


In [8]:
h = tf.assign(d, d + a) # d = d + a, d += a を表現
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5):
        print(sess.run(h)) # 実行ごとに値が更新される

1
2
3
4
5


## 4. プレースホルダー
プレースホルダーを利用すれば実行時に辞書を用いて値を設定できる

In [9]:
p = tf.placeholder(dtype=tf.int32, name='p')
with tf.Session() as sess:
    print('p =', sess.run(p, feed_dict={p: 10}))

p = 10


## 5. 配列とテンソル演算

In [10]:
# ベクトル
a = tf.constant([1, 2, 3], name='a')
b = tf.constant([4, 5, 6], name='b')
c = a + b
with tf.Session() as sess:
    print('a + b =', sess.run(c))

a + b = [5 7 9]


In [11]:
# マトリクス
a = tf.constant([[1, 2], [3, 4]], name='a')
b = tf.constant([[1], [2]], name='b')
c = tf.matmul(a, b)
print(a.shape)
print(b.shape)
print(c.shape)

with tf.Session() as sess:
    print('a =\n', sess.run(a))
    print('b =\n', sess.run(b))
    print('c =\n', sess.run(c))

(2, 2)
(2, 1)
(2, 1)
a =
 [[1 2]
 [3 4]]
b =
 [[1]
 [2]]
c =
 [[ 5]
 [11]]


テンソルでプレースホルダーを用いる場合、大きさが決まっていない次元については `shape=(None, 2)` などと `None` を指定する。

In [12]:
a = tf.placeholder(shape=(None, 2), dtype=tf.int32, name='a')

with tf.Session() as sess:
    print('a =', sess.run(a, feed_dict={a: [[1, 2]]}))
    print('a =', sess.run(a, feed_dict={a: [[1, 2], [3, 4]]}))
                                        

a = [[1 2]]
a = [[1 2]
 [3 4]]


## 6. Saver によるセッション変数の保存
Saver を利用することによって、セッション変数を保存し、他セッションや別プロセスでデータを利用することができる。

In [13]:
a = tf.Variable(1, name='a')
b = tf.assign(a, a + 1)

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(b))
    print(sess.run(b))
    saver.save(sess, '.model/model.ckpt')
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, save_path='.model/model.ckpt')
    print(sess.run(b))
    print(sess.run(b))

2
3
INFO:tensorflow:Restoring parameters from .model/model.ckpt
4
5


## 7. 勾配法

In [16]:
## 最急降下法

# パラメーターは変数として定義
x = tf.Variable(0., name='x')

# 最小化したい関数を定義
func = (x - 1)**2

# オプティマイザーの定義
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate = 0.1 # learning_rate は一度にずらす大きさ
)

# ずらす捜査を train_step として定義
train_step = optimizer.minimize(func)

# 最適化を行う
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20):
        sess.run(train_step)
        print('x =', sess.run(x))


x = 0.2
x = 0.36
x = 0.48800004
x = 0.59040004
x = 0.67232
x = 0.73785603
x = 0.7902848
x = 0.8322278
x = 0.86578226
x = 0.8926258
x = 0.91410065
x = 0.9312805
x = 0.9450244
x = 0.9560195
x = 0.9648156
x = 0.9718525
x = 0.97748196
x = 0.98198557
x = 0.98558843
x = 0.98847073
